At change the resolution, I modifie the beam FWHM in ALMA (my data). Due the beam follows a Gaussian distribution, so the new FWHM must be calculated correctly when convolving. I need to convolve the cube with a Gaussian beam that has a size equivalent to the quadratic difference between the two resolutions.

$FWHM_{conv} = \sqrt{FWHM_{target}^2 - FWHM_{current}^2}$

where:
- $FWHM_{target} = 16"$
- $FWHM_{current} = 14"$ (our data) ---este no es el fwhm

# buscar kernel, ve4r como se hace la convolucion, el minimo que crece es 1.17

#### Devoncolution
${Beam_{target}}^2 = {Beam_{current}}^2 + {Beam_{convolution}}^2$ 

De esta ecuación, puedes despejar el haz de convolución (${Beam_{convolution}}$):

${Beam_{convolution}}^2 = {Beam_{target}}^2 - {Beam_{current}}^2 $ revisar esto!!!

In [ ]:
##### Resampling the spatial resolution - 14" to 16"
from astropy.convolution import convolve, Gaussian2DKernel
from radio_beam import Beam

# Load the cube
cube = SpectralCube.read("CUBE_RESAMPLED_SPECTRAL.fits")

# Obtener el haz actual (resolución espacial actual)
current_beam = cube.beam
Bmaj = current_beam.major.to(u.arcsec)
Bmin = current_beam.minor.to(u.arcsec)
print(Bmaj, Bmin)

# Definir la nueva resolución espacial deseada
BNEWmaj = 16 * u.arcsec
BNEWmin = 16 * u.arcsec


# Calcular el haz de convolución necesario
convolution_beam = Beam(major=BNEWmaj, minor=BNEWmin, pa=current_beam.pa)  #pa = position angle 
convolution_kernel = convolution_beam.deconvolve(current_beam) #calcula el haz gaussiano que necesito aplicar (para producir el haz obj)

# Convolucionar el cubo para alcanzar la nueva resolución
convolved_cube = cube.convolve_to(convolution_beam)

print(convolved_cube)

15.9921123122052 arcsec 11.6163975211884 arcsec
SpectralCube with shape=(729, 128, 120) and unit=Jy / beam:
 n_x:    120  type_x: RA---SIN  unit_x: deg    range:    84.794668 deg:   85.014927 deg
 n_y:    128  type_y: DEC--SIN  unit_y: deg    range:   -69.807597 deg:  -69.726457 deg
 n_s:    729  type_s: VRAD      unit_s: km / s  range:     -129.445 km / s:     598.555 km / s


In [ ]:

# Definir la nueva resolución espacial deseada
BNEWmaj = 16 * u.arcsec
BNEWmin = 16 * u.arcsec


# Calcular el haz de convolución necesario
convolution_beam = Beam(major=BNEWmaj, minor=BNEWmin, pa=current_beam.pa)  #pa = position angle 
convolution_kernel = convolution_beam.deconvolve(current_beam) #calcula el haz gaussiano que necesito aplicar (para producir el haz obj)

# Convolucionar el cubo para alcanzar la nueva resolución
convolved_cube = cube.convolve_to(convolution_beam)

print(convolved_cube)

In [ ]:
convolved_cube.write("CUBE_RESAMPLED_SPATIAL.fits", overwrite=True)

print("El cubo ha sido resampleado a una resolución espacial de 16\" y guardado como 'CUBE_RESAMPLED_SPATIAL.fits'.")

cube_new = SpectralCube.read("CUBE_RESAMPLED_SPATIAL.fits")

# Revisar beam
print(f"Nuevo beam: {cube_new.beam.major.to(u.arcsec):.2f} x {cube_new.beam.minor.to(u.arcsec):.2f} arcsec")

In [ ]:
sub_cube = sub_cube.with_spectral_unit(u.Angstrom, velocity_convention='optical', rest_value=line_wave * u.Angstrom)
#
import scipy
import scipy.constants       as sc
from   scipy                 import constants
from   scipy                 import stats # collection of mathematical algorithms and convenience functions built on the NumPy extension of Python
from   scipy.optimize        import curve_fit # provides functions for minimizing (or maximizing) objective functions, possibly subject to constraints. It includes solvers for nonlinear problems (with support for both local and global optimization algorithms), linear programing, constrained and nonlinear least-squares, root finding, and curve fitting.
from   scipy.optimize        import minimize # functions for minimizing (or maximizing) objective functions, possibly subject to constraints
from   scipy.optimize        import fsolve # Find the roots of a function.
from   scipy.stats           import gaussian_kde # for density plots

# flux density units are not recognized by SpectralCube, so they have to be defined
u.add_enabled_units(u.def_unit(['ergs/s/cm^2/A'], represents=u.erg / u.s /u.cm**2 / u.Angstrom)) 
u.add_enabled_units(u.def_unit(['K (Tmb)'], represents=u.Kelvin))
u.add_enabled_units(u.def_unit(['K (Ta*)'], represents=u.Kelvin))


###Esto es para Jy/beam a K?